# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance II de Proyecto

## Ingeniería de Características

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [28]:
# Librerias
import pandas as pd
import yaml
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

import warnings
# Ignora solo los avisos de funciones que van a cambiar en el futuro
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importando modulos
import sys
from pathlib import Path
# ----------------- Esto solo es para importar src de un folder antes
parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from src import functions as f
from src import modelsF as mf
from src import filtering_vars as fv

> # Generación de Nuevas Características

En esta etapa, realizaremos la estructuración y generación de nuevas variables basadas en el diseño experimental del estudio. El proceso se divide en dos ejes fundamentales:

- **Segmentación Temporal y de Grupo:** Identificación de los periodos de Intervención y Control para cada individuo, asignando etiquetas cronológicas (Pre y Post) a cada medición.

In [29]:
# Cargando Base de Datos
df = pd.read_excel(r"../Data/Final_DF.xlsx")

In [30]:
# Creando columnas identificadoras
df['Treatment'] = np.where(df['tratamiento'] == 1 , 'Intervencion', 'Control')
df['Time'] = np.where(df['tiempo'] == 1 , 'Pre', 'Post')

# Eliminando otras variables identificadoras
df = df.drop(['visita', 'periodo', 'tiempo', 'tratamiento'], axis='columns')

In [31]:
# Infiriendo tipo de variables
uv_cols = list(set(df.columns) - set(['id','grupo','Treatment','Time']))

type_cols = fv.infer_var_types(df,uv_cols)
# Creando un Dataframe con el tipo de variables
df_var_types = pd.DataFrame(type_cols.items(), columns=['Variable','Tipo'])

continuous_vars = list(df_var_types[df_var_types['Tipo']== 'Numerica']['Variable'])

Se realizó un proceso de Ingeniería de Características sobre las variables temporales *(pgh_h_acostarse, pgh_h_levantarse)* para resolver la discontinuidad numérica que presenta el ciclo de 24 horas.

Dado que el modelo debe interpretar que la 01:00 AM es un horario 'posterior' a las 23:00 PM, transformamos la variable circular en una escala lineal continua (contabilizando horas a partir del mediodía). Esta transformación ordinal preserva la monotonicidad del ciclo sueño-vigilia, permitiendo que el algoritmo capture correctamente la relación entre los hábitos de sueño tardíos (desvelos) y los marcadores de polifenoles, algo que un simple binning o la hora cruda (0-23) no lograría identificar correctamente.

Nota: a los valore de 1 am los dejaremos como 25 para conservar el orden.

In [32]:
# Imprimiendo valore únicos de la variable acostarse
df['pgh_h_acostarse'].unique()

array(['23:30:00', '23:45:00', '23:15:00', '23:00:00', '22:00:00',
       '21:30:00', '22:30:00', '22:45:00', '00:00:00', '00:30:00',
       '01:00:00', '21:15:00', '23:50:00',
       datetime.datetime(1900, 1, 1, 0, 0), '21:45:00', '23:40:00',
       '00:10:00', '20:00:00', '01:01:00', '00:15:00'], dtype=object)

In [33]:
# Transformando a string.
df['pgh_h_acostarse'] = df['pgh_h_acostarse'].astype(str)
# Extrayendo hora, para el caso que lo detectó como date time, nos quedamos con la parte de la derecha.
df['pgh_h_acostarse'] = df['pgh_h_acostarse'].apply(lambda x: x.split(' ')[-1])

# Convirtiendo a hora.
df['pgh_h_acostarse'] = pd.to_datetime(df['pgh_h_acostarse'], format='%H:%M:%S', errors='coerce').dt.hour
# Estableciendo como variable categórica ordinal
df['pgh_h_acostarse'] = np.where(df['pgh_h_acostarse'] < 12, df['pgh_h_acostarse'] +24, df['pgh_h_acostarse'])

In [34]:
# Imprimiendo valore únicos de la variable levantarse
df['pgh_h_levantarse'].unique()

array(['07:00:00', '07:15:00', '06:45:00', '06:15:00', '06:07:00',
       '06:10:00', '06:00:00', '08:00:00', '08:05:00', '07:10:00',
       '06:20:00', '06:30:00', '06:37:00', '06:40:00', '07:30:00',
       '07:05:00', '05:00:00', '05:40:00', '08:30:00', '05:15:00',
       '05:30:00', '09:00:00', '07:20:00', '08:15:00', '10:00:00',
       '11:00:00', '07:55:00', '07:45:00'], dtype=object)

In [35]:
# Convirtiendo a hora y al mismo tiempo a variable categórica ordinal.
df['pgh_h_levantarse'] = pd.to_datetime(df['pgh_h_levantarse'], format='%H:%M:%S', errors='coerce').dt.hour

In [ ]:
# Estandarizando sexo a variable binaria
df['sexo_M'] = np.where(df['sexo'] == 1, 1, 0)
del df['sexo']

# Escalamiento

Se seleccionó la técnica de MinMax Scaler para transformar las variables numéricas a un rango acotado [0, 1]. Esta normalización es crítica para evitar que variables con magnitudes absolutas grandes dominen la función objetivo del modelo, asegurando que todas las características contribuyan equitativamente al aprendizaje. Además, al acotar los valores, se facilita la convergencia de los algoritmos de optimización y se mejora la estabilidad numérica durante el entrenamiento.

In [39]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Creando DF con escalamiento
df_scaled = df.copy()

# Apply MinMaxScaler to the identified numerical features
df_scaled[continuous_vars] = scaler.fit_transform(df_scaled[continuous_vars])

print("MinMaxScaler aplicado a las variables numericas.")
print(f"Dimension del Dataframe: {df_scaled.shape}")
print("Imprimiendo primeros renglones del DF escalado:")
display(df_scaled.head())

MinMaxScaler aplicado a las variables numericas.
Dimension del Dataframe: (168, 522)
Imprimiendo primeros renglones del DF escalado:


,id,grupo,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,puntaje_moca,edad,edo_civil,personas_hogar,...,pgh_disfuncion_dia,pgh_duracion_sueño,pgh_total,cat_pgh_total,total_meds,menst_flag,menst_tipo,Treatment,Time,sexo_M
0,1,1,2,5,0.92,4,29,0.522864,1,4,...,0,2,0.2500,1,2,0,MPM,Intervencion,Pre,0
1,1,1,2,5,0.92,4,29,0.537906,1,4,...,0,0,0.0000,1,0,0,MPM,Intervencion,Post,0
2,1,1,2,5,0.92,4,29,0.543321,1,4,...,1,1,0.1875,1,0,0,MPM,Control,Pre,0
3,1,1,2,5,0.92,4,29,0.558363,1,4,...,0,0,0.0000,1,0,0,MPM,Control,Post,0
4,2,2,2,2,0.60,4,28,0.870036,2,3,...,1,2,0.1875,1,5,0,Hombre,Control,Pre,1


# Transformación Yeo–Johnson

En esta fase de ingeniería de características se generó una versión transformada del conjunto de datos utilizando la transformación de potencia Yeo–Johnson, aplicada a las variables numéricas.

El dataset incluía múltiples tipos de mediciones biomédicas y conductuales, muchas de las cuales presentan distribuciones altamente asimétricas:

* biomarcadores con colas largas
* variables de consumo con muchos ceros
* mediciones fisiológicas con valores extremos naturales

En estos casos, trabajar con los valores originales puede generar:

* alta influencia de outliers
* varianza inestable
* dificultad para aplicar modelos estadísticos interpretables

Por ello, la transformación busca mejorar la estructura estadística del dataset antes del modelado.

In [40]:
print("Variables continuas presentes en df para transformar:", len(continuous_vars))
 
# Copia del dataset
df_transformed = df.copy()
 
# Transformador
pt = PowerTransformer(method="yeo-johnson", standardize=False)

# Aplicar solo a variables continuas
df_transformed[continuous_vars] = pt.fit_transform(df[continuous_vars])
 
print("Transformación Yeo–Johnson aplicada correctamente.")

Variables continuas presentes en df para transformar: 371
Transformación Yeo–Johnson aplicada correctamente.


In [11]:
col = continuous_vars[0]
 
print('Comparacion antes vs despues de la transformacion. \n')
print("Antes:")
print(df[col].describe())
 
print("\nDespués:")
print(df_transformed[col].describe())

Comparacion antes vs despues de la transformacion. 

Antes:
count    168.000000
mean       8.222222
std       12.209453
min        0.000000
25%        0.000000
50%        9.333333
75%        9.333333
max       60.000000
Name: ffq_esparragos, dtype: float64

Después:
count    168.000000
mean       1.213123
std        1.190842
min       -0.000000
25%       -0.000000
50%        2.080863
75%        2.080863
max        3.365581
Name: ffq_esparragos, dtype: float64


In [12]:
df_transformed.to_excel("../data/MITOS_YeoJohnson_continuas.xlsx", index=False)
 
print("Archivo guardado: MITOS_YeoJohnson_continuas.xlsx")

Archivo guardado: MITOS_YeoJohnson_continuas.xlsx


# Reducción De Dimensionalidad

No se realizará ninguna de estas técnicas, la justificación será la siguiente:

## **PCA**
Técnicamente: Realiza una transformación lineal ortogonal para proyectar los datos en un nuevo espacio donde la varianza se maximiza. Las nuevas características (PC1, PC2...) son combinaciones lineales de todas las variables originales.

Por qué no: El "Componente 1" podría ser 0.5*Edad - 0.2*Glucosa + 0.1*Peso. Clínicamente, esto es casi imposible de traducir en un perfil de paciente accionable.

## ONE HOT ENCODER

Se optó por la codificación One-Hot (OneHotEncoder) debido a la naturaleza estrictamente nominal de nuestras variables categóricas. A diferencia de otras técnicas que asignan valores secuenciales (1, 2, 3...), el One-Hot genera vectores binarios ortogonales. Esto es crucial para evitar que los algoritmos de aprendizaje automático interpreten erróneamente un **orden de magnitud o jerarquía** inexistente entre las categorías, lo cual introduciría sesgos matemáticos falsos en las predicciones.

Somos conscientes de la posible multicolinealidad. Si el modelo seleccionado es sensible a esto (como una regresión lineal estándar), utilizaremos el parámetro *drop='first'* en el encoder o aplicaremos regularización para mitigar la redundancia de información.

In [41]:
# Seleccionando variables categóricas.
categorical_features_to_encode = list(df_var_types[df_var_types['Tipo']== 'Categorica']['Variable'])

# Aplicando One-Hot encoding.
df_encoded_features = pd.get_dummies(df[categorical_features_to_encode], drop_first=True).astype(int)

# Eliminando las variables categóricas originales.
df = df.drop(columns=categorical_features_to_encode)

# Uniendo al DataFrame Original.
df = pd.concat([df, df_encoded_features], axis=1)

print("One-hot encoding aplicado a variables categoricas.")
print(f"Nueva dimensión del DataFrame: {df.shape}")
print("Primeros 5 renglones del DataFrame:")
display(df.head())

One-hot encoding aplicado a variables categoricas.
Nueva dimensión del DataFrame: (168, 540)
Primeros 5 renglones del DataFrame:


,id,grupo,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,puntaje_moca,edad,edo_civil,personas_hogar,...,num_depo_Dos + Por Día,num_depo_Una Por Día,leucocitos_sed_(1-3),leucocitos_sed_(100-150),leucocitos_sed_(25-30),leucocitos_sed_(3-5),leucocitos_sed_(35-60),leucocitos_sed_(5-10),leucocitos_sed_Moderado,leucocitos_sed_Negativo
0,1,1,2,5,29,4,29,48.71,1,4,...,0,1,0,0,0,0,0,0,0,1
1,1,1,2,5,29,4,29,48.96,1,4,...,0,1,0,0,0,0,0,0,1,0
2,1,1,2,5,29,4,29,49.05,1,4,...,1,0,0,1,0,0,0,0,0,0
3,1,1,2,5,29,4,29,49.30,1,4,...,1,0,0,0,0,0,0,0,0,1
4,2,2,2,2,21,4,28,54.48,2,3,...,0,0,0,0,0,0,0,0,0,1


In [42]:
df.to_csv('../Data/DF_Final_No_Filtered.csv',index=False, encoding='utf-8')

> # Tecnicas De Filtrado

# Umbral De Varianza

El algoritmo implementado (`variance_filter`) es una técnica de **selección de características no supervisada**. Su objetivo es identificar y descartar variables que aportan poca o nula información al modelo (baja varianza) o que tienen una distribución extremadamente desbalanceada.

La estrategia se adapta dinámicamente según el tipo de variable (Numérica, Ordinal, Binaria o Categórica).

### 1. Preprocesamiento: Transformación Z-Score
Para las variables numéricas, el código utiliza una función auxiliar `zscore_df`.
* **Lógica:** $Z = \frac{x - \mu}{\sigma}$
* **Objetivo:** Estandarizar las variables para que tengan una media de 0 y una desviación estándar de 1.
* **Importancia:** Esto elimina la escala de las variables originales. Sin esto, una variable con unidades pequeñas (ej. 0.0001) podría parecer que tiene "baja varianza" comparada con una de unidades grandes (ej. 1000), aunque ambas sean informativas.

### 2. Reglas de Filtrado por Tipo de Variable

A continuación se describe el criterio de descarte para cada tipo de dato:

#### A. Variables Numéricas (`Numerica`)
Se busca eliminar variables **casi-constantes**.
1.  **Transformación:** Se aplica el `zscore_df` a la columna.
2.  **Cálculo:** Se calcula la varianza de los valores Z (normalizados).
3.  **Criterio:**
    * Se conserva si: `Varianza(Z) >= continuous_var_threshold` (Default: 0.01).
    * *Interpretación:* Si la varianza de la variable normalizada es cercana a 0, significa que la variable original es prácticamente un valor único repetido.

#### B. Variables Ordinales (`Ordinal`)
Se asume que una variable ordinal debe tener suficiente granularidad para establecer un orden significativo.
1.  **Cálculo:** Se cuentan los valores únicos (`nunique`).
2.  **Criterio:**
    * Se conserva si: `nunique >= ordinal_min_unique` (Default: 3).
    * *Interpretación:* Si una variable ordinal tiene menos de 3 niveles, se considera que tiene poca capacidad para diferenciar rangos y podría tratarse mejor como binaria o descartarse.

#### C. Variables Binarias (`Binaria`)
Se busca eliminar variables con **baja varianza debido a desbalance extremo**.
1.  **Cálculo:** Se obtiene la frecuencia relativa de la clase más común (`top_freq`).
2.  **Derivación:** Se calcula la frecuencia de la clase minoritaria: $Minoría = 1.0 - top\_freq$.
3.  **Criterio:**
    * Se conserva si: `Minoría >= binary_minority_freq` (Default: 0.05 o 5%).
    * *Interpretación:* Si el 95% o más de los datos pertenecen a una sola clase (ej. "Sí"), la variable aporta muy poca información discriminativa y puede sesgar el modelo.

#### D. Variables Categóricas (`Categorica` / Nominales)
Se busca eliminar variables constantes (un solo valor).
1.  **Cálculo:** Se cuentan los valores únicos.
2.  **Criterio:**
    * Se conserva si: `nunique >= 2`.
    * *Interpretación:* Si solo existe 1 categoría para todas las filas, la variable es una constante y no sirve para predecir nada.

### Resumen de Parámetros

| Parámetro | Valor Default | Descripción |
| :--- | :--- | :--- |
| `continuous_var_threshold` | `0.01` | Umbral mínimo de varianza para numéricas (post-zscore). |
| `binary_minority_freq` | `0.05` | Porcentaje mínimo que debe representar la clase minoritaria. |
| `ordinal_min_unique` | `3` | Cantidad mínima de niveles para considerar una variable ordinal válida. |

In [24]:
# Dado que por la manipulación de variables, han cambiado, inferiremos una vez más el tipo de variable
# Seleccionando columnas para filtrado, excluyendo las identificadoras
uv_cols = list(set(df.columns) - set(['id','grupo','Treatment','Time']))
# Infiriendo tipo de variables
type_cols = fv.infer_var_types(df,uv_cols)

In [27]:
# Aplicando Análisis de Umbral de varianza
variables_ok, df_reporte_var = fv.variance_filter(df,uv_cols,type_cols)

print(f"Se tienen {df_reporte_var[df_reporte_var['keep'] == False].shape[0]} variables candidatas a eliminar.")
df_reporte_var.to_excel('UdeBarcelona/umbral_varianza.xlsx', index= False)

Se tienen 28 variables candidatas a eliminar.


### Debido a que las decisiones de eliminar variables se deben de discutir con los expertos, se discutirá el reporte con ellos.
### Se discutirán las 35 variables con los expertos.
---

# Correlaciones
# Técnica de Selección de Características: Filtro de Redundancia Basado en Relevancia (BDNF)

algoritmo de selección de variables diseñado para optimizar el conjunto de predictores en el estudio de polifenoles y BDNF.

## 1. Descripción de la Técnica
La función `correlation_filter_bdnf` actúa como un filtro inteligente que reduce la dimensionalidad del dataset bajo dos principios:

1.  **Eliminación de Redundancia (Multicolinealidad):** Detecta pares de variables con una correlación de Spearman $\geq 0.90$. Una correlación tan alta indica que ambas variables aportan prácticamente la misma información.
2.  **Criterio de Relevancia Biológica:** En lugar de eliminar una variable al azar, el algoritmo calcula la correlación de cada una contra el **BDNF** (Variable Objetivo). Se conserva aquella que tenga el coeficiente de correlación más alto, asegurando que el modelo final mantenga los predictores con mayor potencial explicativo.
3.  **Calidad de Datos:** Si no existe una diferencia clara en la relevancia, se selecciona la variable con menos valores faltantes (*NaNs*).

---

## 2. Justificación: ¿Por qué aplicar el filtro en el tiempo "Pre"?

El diseño del experimento cuenta con 4 registros por individuo (Tratamiento/Control en tiempos Pre/Post). Es metodológicamente crucial que la selección de variables se realice **exclusivamente con los datos del tiempo "Pre"**.

### A. Caracterización de la Línea Base
En el estado **Pre**, las variables reflejan la fisiología basal y la homeostasis natural del individuo. Las correlaciones aquí presentes son "puras", ya que no han sido alteradas por la intervención de polifenoles o el efecto del tiempo en el estudio.

### B. Consistencia en Medidas Repetidas
Para comparar el efecto de la intervención, debemos medir exactamente las mismas variables en el tiempo Pre y en el Post. Seleccionar las "mejores" variables en el Pre garantiza que evaluaremos el cambio ($\Delta$) sobre una estructura biológica robusta y predefinida.

---

## 3. Resumen de Reglas de Decisión

| Criterio | Condición | Acción |
| :--- | :--- | :--- |
| **Filtro de Redundancia** | $r \geq 0.90$ | Identificar par de variables redundantes. |
| **Prioridad por Target** | $r(A, BDNF) > r(B, BDNF)$ | Mantener **A**, eliminar **B**. |
| **Punto de Corte** | Tiempo == "Pre" | Filtrar dataset para todo el análisis de correlaciones. |

In [16]:
variables_ok_cor, df_reporte_var_cor = fv.correlation_filter_bdnf(df[df['Time'] == 'Pre'],uv_cols,type_cols)

print(f"Se tienen {df_reporte_var_cor.shape[0]} variables candidatas a eliminar.")
df_reporte_var_cor.to_excel('UdeBarcelona/correlaciones.xlsx', index= False)

Se tienen 53 variables candidatas a eliminar.


### Debido a que las decisiones de eliminar variables se deben de discutir con los expertos, se discutirá el reporte con ellos.
### Se discutirán las 56 variables con los expertos.
---

---
# Análisis: Modelo Lineal Mixto (LMM) y Difference-in-Differences (DiD)
Este análisis utiliza un **Modelo Lineal Mixto (Mixed Linear Model - LMM)**. Dado el diseño del ensayo clínico (mediciones repetidas en los mismos sujetos), este enfoque es superior a un ANOVA simple o pruebas t, ya que modela explícitamente la correlación intra-sujeto. El propósito principal de este análisis será identificar variables con mucha importancia con un modelo relativamente sencillo, si bien no decidiremos eliminar ninguna variable con este análisis, si podremos decidir cuáles tenemos que conservar.

## Fundamento Teórico: Diferencia en Diferencias (DiD)

El objetivo central del estudio no es solo determinar si un grupo es mejor que otro en promedio, ni si los pacientes mejoran simplemente por el paso del tiempo. El objetivo es determinar si la **tasa de mejora** es distinta entre los grupos.

## Diseño
- N = 42 sujetos (ID)
- Entre-sujetos: `Group` (1 vs 2)
- Intra-sujeto (repetidas): 
  - `Treatment` (Intervencion vs Control)
  - `Time` (Pre vs Post)
- Cada sujeto aporta 4 observaciones: (Treatment × Time)

Este diseño induce correlación intra-sujeto que invalida pruebas independientes simples.

## Modelo
Para cada variable numérica Y:

Y ~ Group * Treatment * Time + (1 | ID)

Donde:
- `Group`, `Treatment`, `Time` son efectos fijos
- `(1|ID)` es intercepto aleatorio (captura heterogeneidad basal por sujeto)

## Interpretación DiD
- La interacción `Treatment:Time` representa el efecto Difference-in-Differences (cambio pre→post en Intervention menos cambio pre→post en Control) para el grupo de referencia.
- La interacción triple `Group:Treatment:Time` evalúa si el efecto DiD difiere entre Group 1 y Group 2.

Conceptualmente, buscamos la interacción estadística conocida como "Difference-in-Differences":

$$\text{Efecto Neto} = (\Delta_{\text{Grupo Experimental}}) - (\Delta_{\text{Grupo Control}})$$

$$\text{Efecto} = (\bar{X}_{A, \text{Post}} - \bar{X}_{A, \text{Pre}}) - (\bar{X}_{B, \text{Post}} - \bar{X}_{B, \text{Pre}})$$

## Salidas reportadas
- Coeficientes (betas), errores estándar, IC95%, p-values
- Ajuste por comparaciones múltiples (FDR) a través de variables (200+)
- Efectos marginales (contrastes) para estimar DiD por grupo

## Ventajas
- Modela explícitamente la estructura de medidas repetidas
- Usa toda la información sin colapsar datos a deltas
- Es más robusto que t-tests/ANOVA simple en presencia de correlación intra-sujeto

In [17]:
# Preparando el DataFrame
# Excluiremos las variables deltas, por la naturaleza de esas mediciones.
working_df, numeric_vars = f.prepare_dataframe(df, exclude_cols= config['Tratamiento_Variables']['Objetivos'][1:])

working_vars = ['id','grupo','Treatment','Time'] + numeric_vars

working_df = working_df[working_vars]
print(f"Realizaremos este primer análisis utilizando {len(numeric_vars) - 1} variables numéricas.")

Realizaremos este primer análisis utilizando 211 variables numéricas.


>> ## Ajuste de un LMM por variable

In [18]:
results, failed = mf.run_lmm_screen(
    working_df,
    id_col="id",
    group_col="grupo",
    treatment_col="Treatment",
    time_col="Time"
)

c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\sta

### Interpretando hallazgos

In [19]:
# Interpretando
df_interpretado = f.interpretar_hallazgos_final(results)

# Filtrar solo lo relevante para mostrar
cols_reporte = [
    "variable",                # Biomarcador
    "Interpretacion",          # Tu conclusión verbal
    
    # Magnitud del Efecto Clínico (Estimaciones Netas)
    "Efecto_Suplemento_G1",    # ¿Cuánto mejoró el Grupo 1?
    "Efecto_Suplemento_G2",    # ¿Cuánto mejoró el Grupo 2?
    
    # Desglose Estadístico (Evidencia)
    "Triple_q_FDR",            # ¿Es diferente entre grupos? (q-value ajustado)
    "DiD_q_FDR",               # ¿Es efecto del tratamiento? (q-value ajustado)
    
    # Componentes Crudos (Para referencia técnica si te preguntan)
    "Triple_beta",             # El diferencial puro
    "Variacion_Natural"        # El efecto tiempo/aprendizaje (Time_beta)
]

# Mostrar solo las variables donde hubo ALGÚN hallazgo (Prioridad 1 o 2)
hallazgos_significativos = df_interpretado[df_interpretado["Ranking"] < 3][cols_reporte]

print(f"Se encontraron {len(hallazgos_significativos)} biomarcadores con respuesta significativa.")

Se encontraron 26 biomarcadores con respuesta significativa.


In [20]:
hallazgos_significativos.to_excel('../Entregables/hallazgos_LMM.xlsx', index= False)

# Conclusión

En la fase actual del proyecto de polifenoles, se ha completado con éxito la transformación y escalamiento se aplicó generación de nuevas características, binning, codificación one hot, escalamiento min-max, transformación yeo johnson y se justifica nuestros resultados en cada etapa.

La transformación Yeo–Johnson permitió reducir la asimetría y estabilizar la varianza de estas variables continuas, generando un conjunto de datos más adecuado para análisis estadísticos y modelos multivariados posteriores. A diferencia de otras transformaciones como Box–Cox o logaritmos, Yeo–Johnson fue seleccionada debido a su flexibilidad para manejar valores cero o negativos, comunes en mediciones fisiológicas y escalas clínicas.

De forma paralela, se ejecutó un análisis riguroso mediante Modelos Lineales Mixtos (LMM) y Diferencias en Diferencias (DiD). El objetivo principal fue interpretar los resultados para generar conclusiones sólidas y, crucialmente, identificar las variables de mayor relevancia para protegerlas de una eliminación prematura.

Por otro lado, aunque las técnicas de umbral de varianza y correlaciones ya han señalado las candidatas a desechar, hemos decidido pausar su eliminación. La depuración final se ejecutará únicamente tras validar la coherencia de estos hallazgos con los expertos de Barcelona.